In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install finance-datareader
!pip install neuralprophet[live]

     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 155 kB 37.3 MB/s 
  Attempting uninstall: holidays
    Found existing installation: holidays 0.10.5.2
    Uninstalling holidays-0.10.5.2:
      Successfully uninstalled holidays-0.10.5.2


In [3]:
import numpy as np
import pandas as pd
import gc
import math
import os.path
import time
import matplotlib.pyplot as plt
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm
import copy
from fbprophet import Prophet

from neuralprophet import NeuralProphet
from neuralprophet import set_random_seed 

import warnings
import datetime
import random
import tensorflow as tf
warnings.filterwarnings("ignore")

# PUBLIC

In [4]:
path = '/content/drive/MyDrive/stock_2/'
list_name = 'stock_list.csv'
sample_name = 'sample_submission.csv'

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


In [5]:
import os
fileList = os.listdir('/content/drive/MyDrive/stock_2/train')

In [6]:
companyList = []
for file in fileList:
    companyName = file.split("_")[1]
    companyName = companyName.split('.')[0]
    if companyName != "csv":
        companyList.append(companyName)
companyList = sorted(companyList)
print(companyList)

['000060', '000080', '000100', '000120', '000150', '000240', '000250', '000270', '000660', '000670', '000720', '000810', '000880', '000990', '001230', '001440', '001450', '001740', '002380', '002790', '003000', '003090', '003380', '003410', '003490', '003670', '003800', '004000', '004020', '004170', '004370', '004490', '004800', '004990', '005250', '005290', '005300', '005380', '005385', '005387', '005490', '005830', '005850', '005930', '005935', '005940', '006260', '006280', '006360', '006400', '006650', '006730', '006800', '007070', '007310', '007390', '008560', '008770', '008930', '009150', '009240', '009540', '009830', '009900', '010060', '010120', '010130', '010620', '010780', '010950', '011000', '011070', '011170', '011200', '011210', '011780', '012330', '012450', '012510', '012750', '013120', '013890', '014680', '015750', '015760', '016360', '016380', '017670', '017800', '018260', '018880', '019170', '020150', '021240', '022100', '023530', '024110', '025900', '025980', '026960',

In [7]:
sample_submission = pd.read_csv(path + 'sample_submission.csv')
sample_submission = sample_submission[['Day']]
sample_submission['Day'] =pd.to_datetime(sample_submission['Day'])

In [8]:
for i in tqdm(sorted(companyList)):
    globals()['df_{}'.format(i)] = pd.read_csv('/content/drive/MyDrive/stock_2/train/stock_' + i + ".csv")
    locals()['df_{}'.format(i)] = locals()['df_{}'.format(i)][['Date', 'Close']]
    locals()['df_{}'.format(i)].rename(columns={'Date':'ds'}, inplace=True)
    locals()['df_{}'.format(i)].rename(columns={'Close':'y'}, inplace=True)

100%|██████████| 370/370 [01:59<00:00,  3.09it/s]


In [ ]:
# model = NeuralProphet(
#     n_forecasts=5,
#         n_lags=370,
#         yearly_seasonality=False,
#         weekly_seasonality=False,
#         daily_seasonality=False,
#         batch_size=64,
#         epochs=100,
#         learning_rate=1.0,
# )

# metric = model.fit(df_000060, freq='D', valid_p=0.1, epochs=100)

# future = model.make_future_dataframe(df_000060, periods=7)
# forecast = model.predict(future)

In [9]:
for i in sorted(companyList):
    globals()['model_{}'.format(i)] = NeuralProphet(
        n_forecasts=5,
        n_lags=120,
        yearly_seasonality='auto',
        weekly_seasonality='auto',
        daily_seasonality='auto',
        batch_size=64,
        epochs=100,
        learning_rate=0.1,
        loss_func='mae',
        normalize='auto'
        )

    globals()['metric_{}'.format(i)] = locals()['model_{}'.format(i)].fit(locals()['df_{}'.format(i)],
    freq='D',
    valid_p=0.1,
    epochs=100)

    globals()['future_{}'.format(i)] = locals()['model_{}'.format(i)].make_future_dataframe(locals()['df_{}'.format(i)], periods=5)
    globals()['forecast_{}'.format(i)] = locals()['model_{}'.format(i)].predict(locals()['future_{}'.format(i)])

INFO - (NP.forecaster._handle_missing_data) - 95 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 95 NaN values in column y were auto-imputed.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
Epoch[100/100]: 100%|██████████| 100/100 [00:01<00:00, 54.38it/s, L1Loss=0.0461, MAE=749, MSE=9.62e+5, RegLoss=0]
INFO - (NP.forecaster._handle_missing_data) - 57 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 57 NaN values in column y were auto-imputed.
INFO - (NP.forecaster._handle_missing_data) - 95 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 95 NaN values in column y were auto-imputed.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to overri

In [10]:
forecast_000060

,ds,y,yhat1,residual1,yhat2,residual2,yhat3,residual3,yhat4,residual4,yhat5,residual5,ar1,ar2,ar3,ar4,ar5,trend,season_weekly
0,2021-05-06,20400.000000,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,2021-05-07,20850.000000,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,2021-05-08,20833.333333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,2021-05-09,20816.666667,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,2021-05-10,20800.000000,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,2021-10-30,NaN,29389.8,NaN,31373.1,NaN,29752.8,NaN,32649.2,NaN,30550,NaN,24313,26296.3,24676,27572.4,25473.2,5110.48,-33.6994
178,2021-10-31,NaN,None,NaN,31614.7,NaN,29307.2,NaN,31255.8,NaN,30969.8,NaN,None,26603.5,24296.1,26244.7,25958.7,5037.11,-25.9748
179,2021-11-01,NaN,None,NaN,None,NaN,28776.9,NaN,30218.3,NaN,31805.6,NaN,None,None,24038.9,25480.3,27067.6,4963.73,-225.724
180,2021-11-02,NaN,None,NaN,None,NaN,None,NaN,30113.1,NaN,32247,NaN,None,None,None,25278.2,27412,4890.36,-55.4215


In [11]:
for i in sorted(companyList):
    globals()['list_{}'.format(i)] = []
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat1'][177])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat2'][178])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat3'][179])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat4'][180])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat5'][181])

In [12]:
sample_submission = pd.read_csv(path + 'sample_submission.csv')
sub_public = sample_submission[0:5]
sub_private = sample_submission[5:10]

In [13]:
for i in sorted(companyList):
    sub_public[i] = locals()['list_{}'.format(i)]

In [14]:
import os
fileList = os.listdir('/content/drive/MyDrive/stock_2/test')

In [15]:
companyList = []
for file in fileList:
    companyName = file.split("_")[1]
    companyName = companyName.split('.')[0]
    if companyName != "csv":
        companyList.append(companyName)
companyList = sorted(companyList)
print(companyList)

['000060', '000080', '000100', '000120', '000150', '000240', '000250', '000270', '000660', '000670', '000720', '000810', '000880', '000990', '001230', '001440', '001450', '001740', '002380', '002790', '003000', '003090', '003380', '003410', '003490', '003670', '003800', '004000', '004020', '004170', '004370', '004490', '004800', '004990', '005250', '005290', '005300', '005380', '005385', '005387', '005490', '005830', '005850', '005930', '005935', '005940', '006260', '006280', '006360', '006400', '006650', '006730', '006800', '007070', '007310', '007390', '008560', '008770', '008930', '009150', '009240', '009540', '009830', '009900', '010060', '010120', '010130', '010620', '010780', '010950', '011000', '011070', '011170', '011200', '011210', '011780', '012330', '012450', '012510', '012750', '013120', '013890', '014680', '015750', '015760', '016360', '016380', '017670', '017800', '018260', '018880', '019170', '020150', '021240', '022100', '023530', '024110', '025900', '025980', '026960',

In [16]:
for i in tqdm(sorted(companyList)):
    globals()['df_{}'.format(i)] = pd.read_csv('/content/drive/MyDrive/stock_2/test/stock_' + i + ".csv")
    locals()['df_{}'.format(i)] = locals()['df_{}'.format(i)][['Date', 'Close']]
    locals()['df_{}'.format(i)].rename(columns={'Date':'ds'}, inplace=True)
    locals()['df_{}'.format(i)].rename(columns={'Close':'y'}, inplace=True)

100%|██████████| 370/370 [01:58<00:00,  3.12it/s]


In [17]:
for i in sorted(companyList):
    globals()['model_{}'.format(i)] = NeuralProphet(
        n_forecasts=5,
        n_lags=120,
        yearly_seasonality='auto',
        weekly_seasonality='auto',
        daily_seasonality='auto',
        batch_size=64,
        epochs=100,
        learning_rate=0.1,
        loss_func='mae',
        normalize='auto'
        )

    globals()['metric_{}'.format(i)] = locals()['model_{}'.format(i)].fit(locals()['df_{}'.format(i)],
    freq='D',
    valid_p=0.1,
    epochs=100)

    globals()['future_{}'.format(i)] = locals()['model_{}'.format(i)].make_future_dataframe(locals()['df_{}'.format(i)], periods=5)
    globals()['forecast_{}'.format(i)] = locals()['model_{}'.format(i)].predict(locals()['future_{}'.format(i)])

INFO - (NP.forecaster._handle_missing_data) - 103 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 103 NaN values in column y were auto-imputed.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
Epoch[100/100]: 100%|██████████| 100/100 [00:01<00:00, 56.52it/s, L1Loss=0.0495, MAE=804, MSE=1.08e+6, RegLoss=0]
INFO - (NP.forecaster._handle_missing_data) - 56 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 56 NaN values in column y were auto-imputed.
INFO - (NP.forecaster._handle_missing_data) - 103 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 103 NaN values in column y were auto-imputed.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to ov

In [18]:
forecast_000060

,ds,y,yhat1,residual1,yhat2,residual2,yhat3,residual3,yhat4,residual4,yhat5,residual5,ar1,ar2,ar3,ar4,ar5,trend,season_weekly
0,2021-06-04,17850.000000,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,2021-06-05,17783.333333,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,2021-06-06,17716.666667,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,2021-06-07,17650.000000,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,2021-06-08,17650.000000,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,2021-11-27,NaN,30025.8,NaN,32024.6,NaN,31374.8,NaN,29130.1,NaN,31823.5,NaN,40001.8,42000.5,41350.7,39106.1,41799.5,-9713.61,-262.359
177,2021-11-28,NaN,None,NaN,32834.8,NaN,30646.2,NaN,30073.6,NaN,30357.7,NaN,None,43011,40822.4,40249.8,40533.9,-9873.75,-302.467
178,2021-11-29,NaN,None,NaN,None,NaN,31008.4,NaN,30326.7,NaN,30998.8,NaN,None,None,41284.4,40602.7,41274.9,-10033.9,-242.153
179,2021-11-30,NaN,None,NaN,None,NaN,None,NaN,31455.5,NaN,32947.3,NaN,None,None,None,41622.2,43114,-10194,27.2858


In [19]:
for i in sorted(companyList):
    globals()['list_{}'.format(i)] = []
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat1'][176])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat2'][177])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat3'][178])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat4'][179])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat5'][180])

In [20]:
for i in sorted(companyList):
    sub_private[i] = locals()['list_{}'.format(i)]

In [21]:
sub_total = pd.concat([sub_public, sub_private])

In [23]:
sub_total.to_csv(path + 'neural_prophet_lag_120.csv', index=False)